# Introducing Quantitative NetColoc (QNetColoc)

**Overview:** This notebook demonstrates the implementation of quantitative network colocalization which allows input seed genes to be assigned individual scores prior to propagation. 


**Background:** NetColoc evaluates the extent to which two gene sets are related in network space, i.e. the extent to which they are colocalized in a molecular interaction network. In the original implementation, the genes within each gene set are all assigned an equal starting weight. However, when generating such gene sets, we may have varying confidence about the membership of each gene, or evidence of the varying strength of each gene's impact. QNetColoc, therefore, extends the functionality of NetColoc by allowing the user to weight the inputs accordingly. The key differences in the pipeline are as follows:

1. Gene sets need to be supplied with associated scores. These scores will be normalized as part of the QNetColoc procedure.
2. When assessing the expected heat at each gene within the network, maintenance of degree-score correlations is assessed as part of the degree-matched randomization. 

The outputs of the pipeline remain unchanged and can be analyzed as previously. Additional benchmarking and optimization of QNetColoc can be found in the related manusript: 
>Wright SN, Yang J, Ideker T. Common and rare genetic variants show network convergence for a majority of human traits. *EMBO Reports* (2026)

The intention of this notebook is to demonstrate the usage of QNetColoc. The choice between QNetColoc and the original binary implementation of NetColoc (BNetColoc) for a given study depends on several factors including:
* Availability of quantitative scores or measurements for input genes
* Confidence in the scores scaling with the biological relevance of genes
* Desire to include lower confidence genes with appropriate weighting

Note: it is recommended to use scores directly related to biological measurements. Scores related to annotations (e.g. citations) that are correlated with node degree in biological networks may lead to biased results. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import ndex2
import networkx as nx

In [2]:
from netcoloc.network_colocalization import calculate_expected_overlap, get_p_from_permutation_results
from netcoloc.netprop_zscore import calculate_heat_zscores, calculate_scored_heat_zscores
from netcoloc.netcoloc_utils import Seeds
from netcoloc.netprop import get_normalized_adjacency_matrix, get_individual_heats_matrix

## Step 1: Load scored gene sets

Here we will examine genes associated with liver fat measurement. Taking gene associations from the GWAS Catalog (Accession: GCST90016673), we partitioned them into two non-overlapping sets A and B.

Genes are identified by NCBI gene IDs and scores represent association p-values (which we transform to -log10 transformed values). 

In [3]:
seeds_a = Seeds('/cellar/users/snwright/Git/NetColoc/example_notebooks/EFO_0010821_GCST90016673_A.txt')

In [4]:
seeds_b = Seeds('/cellar/users/snwright/Git/NetColoc/example_notebooks/EFO_0010821_GCST90016673_B.txt')

## Step 2: Source a molecular interaction network and pre-calculate the heat matrix

A benchmarking analysis demonstrates that the runtime required scales with the number of edges (w’) and the number of nodes (w’’). The diffusion parameter, which controls the rate of propagation through the network, may be set in this step. In practice, we have found that results are not dependent on the choice of this parameter, and recommend using the default value of 0.5.

Here we use the PCNet 2.1 network as an example. Note that due to the network size it may take several minutes to construct the required matrices. Saving the results is recommended for repeated runs.

In [5]:
interactome_uuid='e9c574f4-e87a-11ee-9621-005056ae23aa' # for PCNet
ndex_server='public.ndexbio.org'
ndex_user=None
ndex_password=None
G_int = ndex2.create_nice_cx_from_server(
            ndex_server, 
            username=ndex_user, 
            password=ndex_password, 
            uuid=interactome_uuid
        ).to_networkx()
node_labels = {key:int(node['GeneID']) for key, node in G_int.nodes(data=True)}
G = nx.relabel_nodes(G_int, node_labels)
nodes = list(G.nodes)

# print out the numbers of nodes and edges in the interatome for diagnostic purposes:
print('Number of nodes:', len(G.nodes))
print('\nNumber of edges:', len(G.edges))

Number of nodes: 18839

Number of edges: 1749784


In [6]:
node_names = {v:k for k,v in node_labels.items()}

In [13]:
# pre-calculate matrices used for network propagation. this step takes a few minutes, more for denser interactomes
print('\ncalculating w_prime')
w_prime = get_normalized_adjacency_matrix(G, conserve_heat=True)

print('\ncalculating w_double_prime')
w_double_prime = get_individual_heats_matrix(w_prime, .5)


calculating w_prime

calculating w_double_prime


In [14]:
G_degree = dict(G.degree())

## Step 3: Implement QNetColoc

### Prepare the gene scores

For quantitative NetColoc scores must be positive, and in increasing order of importance. Therefore, we perform a negative log10 transform of the association p-values. Other transformation options include log10 and log2 transforms.

Next, we filter the gene sets to only genes present in the selected network. 

And, finally, to give a constant input heat to network propagation, we normalize all scores to have a sum of one. Other normalization options include max, minmax and log. 

In [15]:
seeds_a.transform_scores('neglog10')
seeds_a.filter_seeds_by_network(nodes)
seeds_a.normalize_scores(method='sum')

0/5 seeds not found in network


In [16]:
seeds_b.transform_scores('neglog10')
seeds_b.filter_seeds_by_network(nodes)
seeds_b.normalize_scores(method='sum')

0/4 seeds not found in network


### Perform Quantitative network propagation

In [17]:
z_AQ, F_QA, F_rand_QA = calculate_scored_heat_zscores(w_double_prime, nodes, G_degree, seeds_a.scores, 
                                                        num_reps=1000, minimum_bin_size=20, verbose=False,
                                                      normalize_heat=None, random_seed=None)

100%|██████████| 1000/1000 [02:09<00:00,  7.71it/s]


In [18]:
z_BQ, F_QB, F_rand_QB = calculate_scored_heat_zscores(w_double_prime, nodes, G_degree, seeds_b.scores, 
                                                        num_reps=1000, minimum_bin_size=20, verbose=False,
                                                      normalize_heat=None, random_seed=None)

100%|██████████| 1000/1000 [02:10<00:00,  7.65it/s]


In [19]:
z_Qcoloc = pd.DataFrame({'z_A':z_AQ, 'z_B':z_BQ})
z_Qcoloc = z_Qcoloc.assign(z_AB = z_Qcoloc.z_A * z_Qcoloc.z_B)

### Assess the significance of network colocalization

In [20]:
observed_Qsz, permuted_Qsz = calculate_expected_overlap(z_AQ, z_BQ, z_score_threshold=3, z1_threshold=1,
                                                    z2_threshold=1, num_reps=1000, plot=False, 
                                                    overlap_control="bin", seed1=seeds_a.genes, seed2=seeds_b.genes)

Overlap seed genes: 0


100%|██████████| 1000/1000 [00:12<00:00, 81.23it/s]


In [21]:
print(f'Size of the colocalized network: {observed_Qsz} genes')
print(f'COLOC Score: {observed_Qsz/np.mean(permuted_Qsz):.2f}')
p = get_p_from_permutation_results(observed_Qsz, permuted_Qsz)
print(f'Significance of network colocalization: {p:.2e}')

Size of the colocalized network: 289 genes
COLOC Score: 2.24
Significance of network colocalization: 9.35e-54


In [22]:
print('Top Genes:')
top = z_Qcoloc.sort_values(by='z_AB', ascending=False).iloc[0:5, 2]
top.index = top.index.map(node_names)
top

Top Genes:


MTARC1     46.145030
GPAM       30.866713
TM6SF2     30.155082
PNPLA3     24.112447
PPP1R3B    19.526084
Name: z_AB, dtype: float64

Note: QNetColoc prioritizes TM6SF2 as the most colocalized gene, consistent with the TM6SF2 having one of the strongest association signals.

## Step 4: Implement non-quantitative NetColoc for comparison

Here, the inputs are two lists of seed genes, each of which is assigned the same weight. 

In [23]:
gene_set_a = seeds_a.genes
gene_set_b = seeds_b.genes

### Implement network propagation

In [24]:
print('\nCalculating SetA z-scores: ')
z_A, Fnew_A, Fnew_rand_A = calculate_heat_zscores(w_double_prime, nodes, 
                                                                    dict(G.degree), 
                                                                    gene_set_a, num_reps=1000,
                                                                    minimum_bin_size=100)

z_A = pd.DataFrame({'z':z_A})


Calculating SetA z-scores: 


100%|██████████| 1000/1000 [00:11<00:00, 83.44it/s]


In [25]:
print('\nCalculating SetA z-scores: ')
z_B, Fnew_B, Fnew_rand_B = calculate_heat_zscores(w_double_prime, nodes, 
                                                                    dict(G.degree), 
                                                                    gene_set_b, num_reps=1000,
                                                                    minimum_bin_size=100)

z_B = pd.DataFrame({'z':z_B})


Calculating SetA z-scores: 


100%|██████████| 1000/1000 [00:11<00:00, 85.01it/s]


In [26]:
z_coloc = z_A.join(z_B, lsuffix='_A', rsuffix='_B')
z_coloc = z_coloc.assign(z_AB = z_coloc.z_A * z_coloc.z_B)

In [27]:
print('Top Genes:')
top = z_coloc.sort_values(by='z_AB', ascending=False).iloc[0:5, 2]
top.index = top.index.map(node_names)
top

Top Genes:


MTARC1    72.302288
SUGP1     58.058361
GPAM      54.911657
TM6SF2    48.997318
TMC4      42.418630
Name: z_AB, dtype: float64

### Assess the significance of network colocalization

In [28]:
observed_sz, permuted_sz = calculate_expected_overlap(z_A, z_B, z_score_threshold=3, z1_threshold=1,
                                                    z2_threshold=1, num_reps=1000, plot=False, 
                                                    overlap_control="bin", seed1=gene_set_a, seed2=gene_set_b)

Overlap seed genes: 0


100%|██████████| 1000/1000 [00:12<00:00, 81.01it/s]


In [29]:
print(f'Size of the colocalized network: {observed_sz} genes')
print(f'COLOC Score: {observed_sz/np.mean(permuted_sz):.2f}')
p = get_p_from_permutation_results(observed_sz, permuted_sz)
print(f'Significance of network colocalization: {p:.2e}')

Size of the colocalized network: 438 genes
COLOC Score: 3.13
Significance of network colocalization: 1.09e-158


## Step 5: Compare the colocalized networks

In [30]:
BNetColoc_genes = list(z_coloc[(z_coloc.z_A>1) & (z_coloc.z_B>1) & (z_coloc.z_AB>3)].index.map(node_names).values)
QNetColoc_genes = list(z_Qcoloc[(z_Qcoloc.z_A>1) & (z_Qcoloc.z_B>1) & (z_Qcoloc.z_AB>3)].index.map(node_names).values)

In [31]:
print(f'Size of BNetColoc colocalized network: {len(BNetColoc_genes)}')
print(f'Size of QNetColoc colocalized network: {len(QNetColoc_genes)}')
print(f'Intersection of BNetColoc & QNetColoc: {len(set(BNetColoc_genes).intersection(set(QNetColoc_genes)))}')

Size of BNetColoc colocalized network: 438
Size of QNetColoc colocalized network: 289
Intersection of BNetColoc & QNetColoc: 245


In addition to the strength of network colocalization, we can examine simiarlities and differences in the functional annotations of the colocalized network genes.

In [32]:
from gprofiler import GProfiler
gp = GProfiler("MyToolName/0.1")

### Functional enrichments of the BNetColoc colocalized network

In [33]:
gp_bnet = pd.DataFrame(gp.profile(BNetColoc_genes,significance_threshold_method='fdr',
                                               sources=['REAC', 'HP']))

Human Phenotypes

In [34]:
gp_bnet[gp_bnet.source=='HP'].loc[:, ['name', 'p_value', 'intersection_size', 'query_size', 'term_size' ]].head(5)

,name,p_value,intersection_size,query_size,term_size
6,Abnormal circulating lipid concentration,1.750664e-26,54,135,335
11,Abnormal circulating cholesterol concentration,2.967069e-22,37,135,164
19,Abnormality of lipoprotein cholesterol concent...,1.971128e-17,26,135,93
20,Hyperlipidemia,4.316553e-17,32,135,163
21,Abnormal circulating organic compound concentr...,9.876500e-17,64,135,764


Reactome

In [35]:
gp_bnet[gp_bnet.source=='REAC'].loc[:, ['name', 'p_value', 'intersection_size', 'query_size', 'term_size' ]].head(5)

,name,p_value,intersection_size,query_size,term_size
0,Metabolism of lipids,3.435771e-125,177,314,747
1,Metabolism,4.916662e-93,226,314,2100
2,Glycerophospholipid biosynthesis,5.504335e-51,56,314,128
3,Phospholipid metabolism,7.097422e-42,60,314,211
4,"Plasma lipoprotein assembly, remodeling, and c...",1.179889e-33,35,314,71


### Functional enrichments of the QNetColoc colocalized network

In [36]:
gp_qnet = pd.DataFrame(gp.profile(QNetColoc_genes,significance_threshold_method='fdr',
                                               sources=['REAC', 'HP']))

Human Phenotypes

In [37]:
gp_qnet[gp_qnet.source=='HP'].loc[:, ['name', 'p_value', 'intersection_size', 'query_size', 'term_size' ]].head(5)

,name,p_value,intersection_size,query_size,term_size
10,Abnormal circulating lipid concentration,3.936809e-17,36,88,335
12,Hyperlipidemia,1.103466e-14,25,88,163
14,Abnormal circulating cholesterol concentration,1.117265e-12,23,88,164
15,Hyperlipoproteinemia,1.117265e-12,14,88,40
16,Abnormal LDL cholesterol concentration,3.238256e-12,14,88,44


Reactome Pathways

In [38]:
gp_qnet[gp_qnet.source=='REAC'].loc[:, ['name', 'p_value', 'intersection_size', 'query_size', 'term_size' ]].head(5)

,name,p_value,intersection_size,query_size,term_size
0,Metabolism of lipids,2.208704e-70,109,207,747
1,Metabolism,2.255332e-46,135,207,2100
2,Glycerophospholipid biosynthesis,1.519596e-33,38,207,128
3,Phospholipid metabolism,2.328508e-28,41,207,211
4,Triglyceride metabolism,9.405670e-20,18,207,38


### Genes unique to each colocalized network

The genes only present in the BNetColoc network appear to be more involved with kidney function, while the genes only present in the QNetColoc network appear to be more relevant to lipid metabolism. 

In [39]:
only_B = set(BNetColoc_genes).difference(set(QNetColoc_genes))
only_Q = set(QNetColoc_genes).difference(set(BNetColoc_genes))

#### Genes unique to BNetColoc

In [40]:
gp_onlyB = pd.DataFrame(gp.profile(list(only_B),significance_threshold_method='fdr',
                                               sources=['REAC', 'HP']))

In [41]:
gp_onlyB[gp_onlyB.source=='HP'].loc[:, ['name', 'p_value', 'intersection_size', 'query_size', 'term_size' ]].head(5)

,name,p_value,intersection_size,query_size,term_size
15,Abnormal circulating lipid concentration,0.000002,19,56,335
16,Abnormal circulating cholesterol concentration,0.000002,14,56,164
19,Abnormal circulating organic compound concentr...,0.000003,27,56,764
22,Abnormal circulating metabolite concentration,0.000014,36,56,1445
27,Abnormality of lipoprotein cholesterol concent...,0.000283,9,56,93


In [42]:
gp_onlyB[gp_onlyB.source=='REAC'].loc[:, ['name', 'p_value', 'intersection_size', 'query_size', 'term_size' ]].head(5)

,name,p_value,intersection_size,query_size,term_size
0,Metabolism of lipids,1.839645e-47,70,123,747
1,Metabolism,1.648651e-42,95,123,2100
2,Cholesterol biosynthesis,2.024961e-17,13,123,26
3,Glycerophospholipid biosynthesis,1.180368e-14,19,123,128
4,Retinoid metabolism and transport,5.125799e-14,13,123,44


#### Genes unique to QNetColoc

In [43]:
gp_onlyQ = pd.DataFrame(gp.profile(list(only_Q),significance_threshold_method='fdr',
                                               sources=['REAC', 'HP']))

In [44]:
gp_onlyQ[gp_onlyQ.source=='HP'].loc[:, ['name', 'p_value', 'intersection_size', 'query_size', 'term_size' ]].head(5)

,name,p_value,intersection_size,query_size,term_size


In [45]:
gp_onlyQ[gp_onlyQ.source=='REAC'].loc[:, ['name', 'p_value', 'intersection_size', 'query_size', 'term_size' ]].head(5)

,name,p_value,intersection_size,query_size,term_size
0,Nuclear Envelope Breakdown,0.004759,3,16,47
1,Depolymerization of the Nuclear Lamina,0.009267,2,16,13
2,Mitotic Prophase,0.035511,3,16,134
